In [102]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd

In [103]:
# Read Data
spectrum_train = pd.read_excel('spectrum_train.xlsx')
spectrum_test = pd.read_excel('spectrum_valid.xlsx')
temp_train = pd.read_excel('temp_train.xlsx')
temp_test = pd.read_excel('temp_valid.xlsx')

In [104]:
# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

In [105]:
input_data = df_to_tensor(spectrum_train)
output_data = df_to_tensor(temp_train)

In [106]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [107]:
input_size = 66
output_size = 11
model = Net(input_size=input_size, output_size=output_size)

In [108]:
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

In [109]:
num_epochs = 100
batch_size = 32
num_batches = len(input_data) // batch_size

for epoch in range(num_epochs):
    for batch in range(num_batches):
        start = batch * batch_size
        end = start + batch_size

        inputs = input_data[start:end]
        targets = output_data[start:end]

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

    # Print the loss for this epoch
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}")

Epoch 10/100, Loss: 2759.895508
Epoch 20/100, Loss: 1840.895630
Epoch 30/100, Loss: 1721.669067
Epoch 40/100, Loss: 1699.722534
Epoch 50/100, Loss: 1708.354248
Epoch 60/100, Loss: 1735.751953
Epoch 70/100, Loss: 1780.304321
Epoch 80/100, Loss: 1842.365112
Epoch 90/100, Loss: 1922.160522
Epoch 100/100, Loss: 1943.137939


In [110]:
predictions = model(df_to_tensor(spectrum_test))

In [111]:
predictions

tensor([[527.8499, 529.1024, 531.7308,  ..., 530.6112, 528.7522, 526.8467],
        [527.8499, 529.1024, 531.7308,  ..., 530.6112, 528.7522, 526.8467],
        [527.8499, 529.1024, 531.7308,  ..., 530.6112, 528.7522, 526.8467],
        ...,
        [527.8499, 529.1024, 531.7308,  ..., 530.6112, 528.7522, 526.8467],
        [527.8499, 529.1024, 531.7308,  ..., 530.6112, 528.7522, 526.8467],
        [527.8499, 529.1024, 531.7308,  ..., 530.6112, 528.7522, 526.8467]],
       grad_fn=<AddmmBackward0>)